In [36]:
from bs4 import BeautifulSoup
import pandas as pd
import os
import numpy as np



In [37]:
file_path = "C:/Users/USER/Desktop/500118.txt"
output_csv_path = 'C:/Users/USER/Desktop/output_data.csv'
test_folder_path="C:/Users/USER/Desktop/testing-data"



In [38]:
file_list = os.listdir(test_folder_path)
soup_objects = []



In [39]:
def get_price(soup):
    car_prices = soup.find_all('h5', class_='post-price')

    if car_prices:
        prices = []

        for price_element in car_prices:
            price = price_element.text.strip().split()[0]
            prices.append(price)

        return prices
    else:
        return None


In [40]:
def get_model(soup):
    driving_table = soup.find("table", class_="driving-table")

    if driving_table:
        car_model_tag = driving_table.find("h3")

        if car_model_tag:
            model = car_model_tag.text.strip()
        else:
            model = None
    else:
        model = None

    return model


In [41]:
def get_year(soup):
    driving_table = soup.find("table", class_="driving-table")

    if driving_table:
        car_year_tag = driving_table.find("h5")

        if car_year_tag:
            car_year_text = car_year_tag.text.strip()
            # Extract numeric part from the text
            numeric_part = ''.join(filter(str.isdigit, car_year_text))
            year = int(numeric_part) if numeric_part else None
        else:
            year = None
    else:
        year = None

    return year


In [42]:

def get_car_attributes(soup):
    car_attributes = {}

    rows = soup.find_all('tr', class_='list-row')

    for row in rows[:-3]:
        columns = row.find_all('td')
    
        if len(columns) == 2:
            attribute = columns[0].text.strip()
            value = columns[1].text.strip()
            car_attributes[attribute] = value

    return car_attributes


In [43]:
prices=[]
models=[]
years=[]
attributes=[]
for file_name in file_list:
    file_path = os.path.join(test_folder_path, file_name)
    
    with open(file_path, 'r', encoding='utf-8') as file:
        html_content = file.read()
        soup = BeautifulSoup(html_content, 'html.parser')
        soup_objects.append(soup)
        price = get_price(soup)
        if price:
            prices.append(price)   
        else:
            prices.append(np.nan)     
        model=get_model(soup)
        if model :
            models.append(model)
        else:
            models.append(np.nan)        
        year=get_year(soup)
        if year:
            years.append(year)
        else :
            years.append(np.nan)    
            

In [44]:
data = {
    "Price": prices,
    "Model": models,
    "Year": years,
}

df = pd.DataFrame(data)
df.to_csv(output_csv_path, index=False, encoding='utf-16')
df

,Price,Model,Year
0,[100000],كيا اوبتيما,2014.0
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,[60000],كيا سورينتو,2007.0
4,[43500],هونداي افانتي,2006.0
5,[2600],هوندا اكورد,1997.0
6,[1800],اوبل اسكونا,1967.0
7,NaN,NaN,NaN
8,[77000],هونداي سانتافيه,2008.0
9,NaN,NaN,NaN
